In [46]:
import sys
from pyspark.sql import functions as F
from pyspark.sql.functions import * 
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from functools import reduce


sys.argv += ['--JOB_NAME','Custom-Translation-Trucape-Invoices-2']
sys.argv += ['--source_csv','s3://sol-dev-source/TruCape-Invoices']
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'source_csv'])
sc = SparkContext.getOrCreate()
gc = GlueContext(sc)
spark = gc.spark_session
job = Job(gc)
job.init(args["JOB_NAME"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 11)
  File "<stdin>", line 11
    import org.apache.spark.sql.{SaveMode, SparkSession}
                                ^
SyntaxError: invalid syntax



In [23]:
# source_dyf=gc.create_dynamic_frame.from_catalog(
#     database="s3",
#     table_name=args["source_csv"],
#     transformations_ctx="source_dyf"
# )

source_dyf = gc.create_dynamic_frame.from_options(
    's3',
    {
        "paths": [
            args["source_csv"]
        ],
    },
    "csv",
    transformation_ctx = "source_dyf",
    headerText = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
source_df = source_dyf.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
source_df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+
|              col0|  col1|               col2|        col3|    col4|         col5|        col6|    col7|      col8|
+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+
|External Reference|Number|       Counterparty| Amount Due |Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
|           SC25648|138099|         JSC TANDER|  21,124.80 |     USD|          ZAR|     15.5654|   7-Apr|    21-Jan|
|           SC25646|138127|HORIZON FRESH FRUIT|  17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
|           SC25645|138129|HORIZON FRESH FRUIT|  17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+

In [26]:
# (b) Create array with desired columns
old_columns = source_df.schema.names
new_columns = [
    field.lower().replace("col0", "External Reference")
    .replace("col1", "Number")
    .replace("col2", "Counterparty")
    .replace("col3", "Amount Due")
    .replace("col4", "Currency")
    .replace("col5", "Cost Currency")
    .replace("col6", "Costing Rate")
    .replace("col7", "Due Date")
    .replace("col8", "Issue Date")
    for field in old_columns
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# (c) Overwrite and persist `new_columns`
source_df = reduce(
    lambda source_df, idx: source_df.withColumnRenamed(old_columns[idx], new_columns[idx]),
    range(len(old_columns)),
    source_df,
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+
|External Reference|Number|       Counterparty|  Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+
|External Reference|Number|       Counterparty| Amount Due |Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
|           SC25648|138099|         JSC TANDER|  21,124.80 |     USD|          ZAR|     15.5654|   7-Apr|    21-Jan|
|           SC25646|138127|HORIZON FRESH FRUIT|  17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
|           SC25645|138129|HORIZON FRESH FRUIT|  17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+

In [29]:
pandas_source_df = source_df.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
source_df = pandas_source_df.iloc[1: , :]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
source_df=spark.createDataFrame(source_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
source_df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+
|External Reference|Number|       Counterparty| Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+
|           SC25648|138099|         JSC TANDER| 21,124.80 |     USD|          ZAR|     15.5654|   7-Apr|    21-Jan|
|           SC25646|138127|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
|           SC25645|138129|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+

In [33]:
# Add notes column to dataset
source_df = source_df.withColumn('Notes', lit(None).cast(StringType()))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+-----+
|External Reference|Number|       Counterparty| Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|Notes|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+-----+
|           SC25648|138099|         JSC TANDER| 21,124.80 |     USD|          ZAR|     15.5654|   7-Apr|    21-Jan| null|
|           SC25646|138127|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan| null|
|           SC25645|138129|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan| null|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+-----+

In [35]:
# ADD YEAR TO DATE COLUMNS
due_date="due date"
issue_date="issue date"
year = F.lit("-2022")
due_date_add_year = F.concat(col(due_date),year)
issue_date_add_year = F.concat(col(issue_date),year)
                               
source_df = source_df.withColumn(due_date, due_date_add_year) \
    .withColumn(issue_date, issue_date_add_year)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# SET DECIMAL PLACES
costing_rate="costing rate"
format_costing_rate = format_number(round(costing_rate ,4), 4)
amount_due="amount due"
format_amount_due=format_number(round(amount_due ,2), 2)

source_df = source_df.withColumn(costing_rate, format_costing_rate) \
# amount due gives null value - not sure why


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# FORMAT DATES
source_df = source_df.select(
    col("external reference"), \
    col("number"), \
    col("counterparty"), \
    col("amount due"), \
    col("currency"), \
    col("cost currency"), \
    col("costing rate"), \
    to_date(F.col("due date"), "d-MMM-yyy").alias("due date"), \
    to_date(F.col("issue date"), "d-MMM-yyy").alias("issue date"), \
     col("notes")
) \

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+-----------+--------+-------------+------------+----------+----------+-----+
|external reference|number|       counterparty| amount due|currency|cost currency|costing rate|  due date|issue date|notes|
+------------------+------+-------------------+-----------+--------+-------------+------------+----------+----------+-----+
|           SC25648|138099|         JSC TANDER| 21,124.80 |     USD|          ZAR|     15.5654|2022-04-07|2022-01-21| null|
|           SC25646|138127|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|2022-03-17|2022-01-21| null|
|           SC25645|138129|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|2022-03-17|2022-01-21| null|
+------------------+------+-------------------+-----------+--------+-------------+------------+----------+----------+-----+

In [45]:
# source_df.write.mode(SaveMode.Overwrite).json('s3://sol-dev-output/TruCape-Invoices')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'SaveMode' is not defined
Traceback (most recent call last):
NameError: name 'SaveMode' is not defined



In [ ]:
# source_df.coalesce(1).write.csv(path='s3://sol-dev-output/TruCape-Invoices', mode='overwrite')


In [48]:
# source_df.coalesce(1).write.format("json").mode("overwrite").options(header="true",sep=",").save(path='s3://sol-dev-output/TruCape-Invoices')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
source_df.repartition(1).write.mode('append').json('s3://sol-dev-output/TruCape-Invoices')

myPath = 's3://sol-dev-output/TruCape-Invoices/*'

timeStamp = current_timestamp()

hadoopPath = SparkContext._jvm.org.apache.hadoop.fs.Path(myPath)

hadoopFs = hadoopPath.getFileSystem(SparkContext._jvm.org.apache.hadoop.conf.Configuration())

statuses = hadoopFs.globStatus(hadoopPath)

file_name = [file.getPath().getName() for file in statuses if file.getPath().getName().startswith('part-')][0]

hadoopFs.rename(SparkContext._jvm.org.apache.hadoop.fs.Path(f"s3://sol-dev-output/TruCape-Invoices/{file_name}"), SparkContext._jvm.org.apache.hadoop.fs.Path("s3://sol-dev-output/TruCape-Invoices/TRU_Invoices.json"))




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True